In [5]:
import os
import sys
import time
import json
import urllib
import datetime

from lxml import html

try:
    import requests
except ImportError:
    # Install requests library to jupyter notebook env
    !{sys.executable} -m pip install requests

from requests import Request, Session
    
import logging

def setup_custom_logger(name):
    formatter = logging.Formatter(fmt='%(asctime)s %(levelname)-8s %(message)s',
                                  datefmt='%Y-%m-%d %H:%M:%S')
    handler = logging.FileHandler('log.txt', mode='w')
    handler.setFormatter(formatter)
    screen_handler = logging.StreamHandler(stream=sys.stdout)
    screen_handler.setFormatter(formatter)
    logger = logging.getLogger(name)
    logger.setLevel(logging.DEBUG)
    logger.addHandler(handler)
    logger.addHandler(screen_handler)
    return logger

try:
    logger
except NameError:
    logger = setup_custom_logger('fresh-direct-poller')

In [9]:
# Your Fresh Direct login credentials
USERNAME = 'ganohi9761@govdep5012.com'
PASSWORD = 'correcthorsebatterystaple'

# XXX: See README.md for info on what this is and where/how to find it.
COOKIE_BLOB = '_abck=E1DBA66CD8232B7847CAC1E0C05890EF~0~YAAQnjkrFxFuhFhxAQAA31wZbAOkPBehAKIl+gK1iawJGUP/fZSiy/Td6Hh4vl50oHKvM11M5OKReeCEtnxtPR9qd5Jciu2KYbp/idI5ycubuNvVvDK5GD/qEGPA1lMFFyjTTEqT4nVJXjmuU27L0I9tSfPwfSZbuAK7aHuPQIw05feEQID4dqFW3USqpDkut9skzCwXBq6B7thmc7E4g/K+tTG+Rw9ywVTUvQw7SpNlzbouqnB3mcDIO/eSbhF/8I3q7nikcvwRUxmp7pXFlc6N0Tt3Ta5099YyjrxRpSgSxN5QBoBnubpzVThq4UPOHhvgp/xawjlUCj/i~-1~1-zhFswvjBly-10000-100-3000-1||||~-1; _fbp=fb.1.1586573004489.843794147; _ga=GA1.2.675829157.1586573004; _gid=GA1.2.1329078685.1586573004; RT="sl=6&ss=1586655292047&tt=38619&obo=0&sh=1586656515519%3D6%3A0%3A38619%2C1586655919718%3D5%3A0%3A32090%2C1586655892825%3D4%3A0%3A24758%2C1586655374625%3D3%3A0%3A18702%2C1586655348101%3D2%3A0%3A12796&dm=freshdirect.com&si=05281a11-4ba5-4f51-acac-1383f9f90d2f&se=900&bcn=%2F%2F173c5b09.akstat.io%2F&ld=1586656515519"; bc_pv_end=; com.silverpop.iMA.page_visit=-1454851620:; com.silverpop.iMA.session=8ccb48a4-0368-8113-ccce-6610b03035c4; com.silverpop.iMAWebCookie=c7914a42-36c0-cd01-284c-e87a8386a3e2; _gat=1; _gali=locabar_user_login_link; IR_8453=1586656512282%7C0%7C1586652201829%7C%7C; _br_uid_2=uid%3D1819493847360%3Av%3D12.0%3Ats%3D1586573003924%3Ahc%3D71; _gat_UA-20535945-1=1; _gcl_au=1.1.1198095965.1586573005; _uetsid=_uet1d3f4e3c-df36-1484-492a-1bf7f4e4eb6e; _mibhv=anon-1586573004001-6107720765_3887; _micpn=esp:-1::1586573004001; _spva_id.b7cc=7f1f95ea-9fb3-4bc0-a786-923b0a5fb766.1586573005.9.1586656513.1586649852.1d962803-f279-4070-9a55-0014968a93ce; _spva_ses.b7cc=*; __gads=ID=6dadb1005a45e891-229af7377db70062:T=1586573003:RT=1586656511:S=ALNI_Mb107N98T_B0WQiHQlRzKmfzVds7w; optimizelyEndUserId=oeu1586573001429r0.8512158743306365; FDUser=B632BSCC3IUF1PQHS8Z79MABJ1SOP5KWHYG0JM; JSESSIONID=C780F00F574E19B2497F5F8D9EC15D71; IR_PI=6659826a-7b9f-11ea-8fd7-0e7e9f208ae4%7C1586742316523; ak_bmsc=0BBD4A40AB368DEDF0D5101F52C81400172B3A449D3200002664925E22BAB637~plfD5GTj6w3rOisgy/a8YCdh62iR+TJb6c3vITZJnDVzL8UiQ1wtZPCfqqKzvzExVKk8wVd8fRNWEp+2yITle9N1cN4JFgd9hchuT2po1amt4oaGzY7WbEGiPYgnagowDrrveVTR4k0Tm60IpU3LxShuiVE0VoAn0Mw00S/DLi7P45bH9WffJN9gi7qjedkqtaYozES2jvE1d0QZKKnDJddsCuKcVYmbLZrdI1Y3cKDYxN5stNRTNtqfalcHloa5tS; bm_mi=3AA65996B544A23FB5DFD5E4E93841F7~6gwuIaTmda4zcJzEWnJR3fN72+WmhJ/r1BMHk2ax86RFK7NEcD8i4A5lwykpSLIJyj3MelHtzIkihmEr3+plEXCXgdakbcPXKaEctW7WGKCSJ2z++iy6K5H8ydDwID7dsg4em127TVXMZ12eXFRwLsoY0u3c9NVOOCKfxYk1K0q23S4+hnyG+ZI4+/GV3eiwNNSl18LYEIekwbdIqlWo8y/p2InS71AXIQd8FGKj6phQjfOPcu1G2EW1h0dE2Ib2RNjqdr6PqebdOGBprldH4PZnV8Dim3gzRunFpr9hr5M=; bm_sz=24AB69CD01729137BD103C281381C2A9~YAAQRDorFwhJpWlxAQAAwDfXawd9PamJGneqQI1o+o51rGSDtp442qX0a9by0ud89sLT7NcOyCZzeAtly0FezrtlO7SH+stL7iYwTjTuDoHGydE+Fxup5dPQn0eJ4g+FTj6HQ6RIjb8r7jDBFj2z5s/lzg3ZZjk/rBJTivmik4U+Oq8FYDQD1rREo4wEIv9ukTX7do0=; _hjIncludedInSample=1; _hjid=a04ab3ee-7460-4d11-912b-3cde1c9cc4da; IR_gbd=freshdirect.com'

In [16]:
class Alerter:
    def __init__(self, alert_interval=60):
        self.alert_interval = alert_interval
        self.last_alerted = datetime.datetime.fromtimestamp(0)
        
    def alert(self, message):
        if (datetime.datetime.now() - self.last_alerted).seconds > self.alert_interval:
            self.last_alerted = datetime.datetime.now()
            self.user_alert(message)
        
    def user_alert(self, message):
        pass  # Implement in subclass.
    
    
class TextAlerter(Alerter):
    def user_alert(self, message):
        pass # TODO: implement text messaging here.

        
class FreshDirectClient:
    auth_endpoint = 'https://www.freshdirect.com/api/login/'
    slots_endpoint = 'https://www.freshdirect.com/your_account/delivery_info_avail_slots.jsp'
    request_timeout = 10
    
    def __init__(self, logger):
        self.headers = {
            'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
            'Accept': '*/*',
            'Accept-Language': 'en-us',
            'Accept-Encoding': 'gzip, deflate, br',
            'Host': 'www.freshdirect.com',
            'Origin': 'https://www.freshdirect.com',
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.5 Safari/605.1.15',
            'Connection': 'keep-alive',
            'Referer': 'https://www.freshdirect.com/index.jsp?serviceType=HOME',
            'Content-Length': '95',
            'Cookie': '',
            'X-Requested-With': 'XMLHttpRequest'
        }
        self.logger = logger
        self.session = Session()

        
    def authenticate(self, user_id, password, cookie_blob):
        auth_data = urllib.parse.quote(json.dumps({'userId': user_id, 'password': password}))
        self.headers['Cookie'] = cookie_blob
        res = self.session.post(self.auth_endpoint, data=auth_data, 
                                headers=self.headers, timeout=self.request_timeout)
        self.logger.info(res.status_code)
        
        if res.status_code != 200:
            raise RuntimeError('Bad response from fresh direct: {}'.format(res.status_code))


    def get_delivery_timeslots_html(self):
        res = self.session.get(self.slots_endpoint, headers=self.headers, timeout=10)
        self.logger.info(res.status_code)

        if res.status_code != 200:
            raise RuntimeError('Bad response from fresh direct: {}'.format(res.status_code))
            
        return res.text
    
    
def parse_timeslots(html_string):
    tree = html.fromstring(html_string)
    
    available_slots = []
    for col in range(7):
        day_name = tree.xpath('//*[@id="ts_d{col}_hE_content"]/div[1]/b'.format(col=col))[0].text
        mmm, dd  = tree.xpath('//*[@id="ts_d{col}_hE_content"]/div[2]'.format(col=col))[0].text.split(' ')
        logger.info('{0} {1} {2}'.format(day_name, mmm, dd))

        for row in range(7):
            time_slot = tree.xpath('//*[@id="ts_d{col}_ts{row}_time"]'.format(col=col, row=row))
            if not time_slot:
                continue
                
            time_slot_text = time_slot[0].text
            message = tree.xpath('//*[@id="ts_d{col}_ts{row}_msgE"]/div/div/font'.format(col=col, row=row))
            if not message:
                continue
                
            message_text = message[0].text.replace('\xa0', ' ')  # '\xa0' is a no-break space.
            sold_out = 'SOLD OUT' in message_text
            if not sold_out:
                thing = '{0} {1} {2} @ {3} {4}'.format(day_name, mmm, dd, time_slot_text, message_text)
                available_slots.append(thing)
            logger.info('\t{0}\t{1} {2}\t{3}'.format("****" if not sold_out else "",
                                                     time_slot_text, message_text, 
                                                     "****" if not sold_out else ""))
    return available_slots


def poll_and_alert(client, alerter, poll_interval):
    while True:
        timeslots_html = client.get_delivery_timeslots_html()
        available_slots = parse_timeslots(timeslots_html)
        if available_slots:
            logger.info('***** TIMESLOTS ARE AVAILABLE ******')
            message = 'The following time slots are available on Fresh Direct:\n\n'
            for slot in available_slots:
                message += slot + '\n'
            logger.info(message)
            alerter.alert(message)
        time.sleep(poll_interval)
        
        
def run_main(poll_interval=15, alert_interval=60):
    """ Main entry point.
    
    Args
        poll_interval (int)
            Number of seconds to wait between refreshes.
        
        alert_interval (int)
            Number of seconds to wait between subsequent alerts.
    """
    start_time = datetime.datetime.now()
    
    client = FreshDirectClient(logger)
    client.authenticate(USERNAME, PASSWORD, COOKIE_BLOB)
    
    alerter = TextAlerter(alert_interval)

    try:
        poll_and_alert(client, alerter, poll_interval)
    except KeyboardInterrupt:
        logger.info('Stopped')
    except Exception as e:
        alerter.alert(str(e))
        logger.info(str(e))
    finally:
        end_time = datetime.datetime.now()
        run_time = end_time - start_time
        message = 'Fresh direct poller finished running.'
        alerter.alert(message)
        logger.info(message)

In [ ]:
run_main()

2020-04-11 22:11:38 INFO     200
2020-04-11 22:11:39 INFO     200
2020-04-11 22:11:39 INFO     SUNDAY Apr 12
2020-04-11 22:11:39 INFO     MONDAY Apr 13
2020-04-11 22:11:39 INFO     		6 am - 9 am SOLD OUT	
2020-04-11 22:11:39 INFO     		3 pm - 6 pm SOLD OUT	
2020-04-11 22:11:39 INFO     		6 pm - 9 pm SOLD OUT	
2020-04-11 22:11:39 INFO     TUESDAY Apr 14
2020-04-11 22:11:39 INFO     		6 am - 9 am SOLD OUT	
2020-04-11 22:11:39 INFO     		9 am - 12 pm SOLD OUT	
2020-04-11 22:11:39 INFO     		3 pm - 6 pm SOLD OUT	
2020-04-11 22:11:39 INFO     		6 pm - 9 pm SOLD OUT	
2020-04-11 22:11:40 INFO     WEDNESDAY Apr 15
2020-04-11 22:11:40 INFO     		6 am - 9 am SOLD OUT	
2020-04-11 22:11:40 INFO     		9 am - 12 pm SOLD OUT	
2020-04-11 22:11:40 INFO     		3 pm - 6 pm SOLD OUT	
2020-04-11 22:11:40 INFO     		6 pm - 9 pm SOLD OUT	
2020-04-11 22:11:40 INFO     THURSDAY Apr 16
2020-04-11 22:11:40 INFO     		3 pm - 6 pm SOLD OUT	
2020-04-11 22:11:40 INFO     		6 pm - 9 pm SOLD OUT	
2020-04-11 22:11:40 IN

2020-04-11 22:13:05 INFO     		6 pm - 9 pm SOLD OUT	
2020-04-11 22:13:05 INFO     		9 pm - 11 pm SOLD OUT	
2020-04-11 22:13:05 INFO     FRIDAY Apr 17
2020-04-11 22:13:05 INFO     		6 am - 9 am SOLD OUT	
2020-04-11 22:13:05 INFO     		9 am - 12 pm SOLD OUT	
2020-04-11 22:13:05 INFO     		3 pm - 6 pm SOLD OUT	
2020-04-11 22:13:05 INFO     		6 pm - 9 pm SOLD OUT	
2020-04-11 22:13:05 INFO     SATURDAY Apr 18
2020-04-11 22:13:05 INFO     		9 am - 12 pm SOLD OUT	
2020-04-11 22:13:05 INFO     		3 pm - 6 pm SOLD OUT	
2020-04-11 22:13:05 INFO     		6 pm - 9 pm SOLD OUT	
2020-04-11 22:13:21 INFO     200
2020-04-11 22:13:21 INFO     SUNDAY Apr 12
2020-04-11 22:13:21 INFO     MONDAY Apr 13
2020-04-11 22:13:21 INFO     		6 am - 9 am SOLD OUT	
2020-04-11 22:13:21 INFO     		3 pm - 6 pm SOLD OUT	
2020-04-11 22:13:21 INFO     		6 pm - 9 pm SOLD OUT	
2020-04-11 22:13:21 INFO     TUESDAY Apr 14
2020-04-11 22:13:21 INFO     		6 am - 9 am SOLD OUT	
2020-04-11 22:13:21 INFO     		9 am - 12 pm SOLD OUT	
202

2020-04-11 22:14:46 INFO     		9 am - 12 pm SOLD OUT	
2020-04-11 22:14:46 INFO     		3 pm - 6 pm SOLD OUT	
2020-04-11 22:14:46 INFO     		6 pm - 9 pm SOLD OUT	
2020-04-11 22:14:46 INFO     TUESDAY Apr 14
2020-04-11 22:14:46 INFO     		6 am - 9 am SOLD OUT	
2020-04-11 22:14:46 INFO     		9 am - 12 pm SOLD OUT	
2020-04-11 22:14:46 INFO     		3 pm - 6 pm SOLD OUT	
2020-04-11 22:14:46 INFO     		6 pm - 9 pm SOLD OUT	
2020-04-11 22:14:46 INFO     WEDNESDAY Apr 15
2020-04-11 22:14:46 INFO     		6 am - 9 am SOLD OUT	
2020-04-11 22:14:46 INFO     		9 am - 12 pm SOLD OUT	
2020-04-11 22:14:46 INFO     		3 pm - 6 pm SOLD OUT	
2020-04-11 22:14:46 INFO     		6 pm - 9 pm SOLD OUT	
2020-04-11 22:14:46 INFO     THURSDAY Apr 16
2020-04-11 22:14:46 INFO     		3 pm - 6 pm SOLD OUT	
2020-04-11 22:14:46 INFO     		6 pm - 9 pm SOLD OUT	
2020-04-11 22:14:46 INFO     		9 pm - 11 pm SOLD OUT	
2020-04-11 22:14:46 INFO     FRIDAY Apr 17
2020-04-11 22:14:46 INFO     		6 am - 9 am SOLD OUT	
2020-04-11 22:14:46 INF

2020-04-11 22:16:11 INFO     		6 pm - 9 pm SOLD OUT	
2020-04-11 22:16:11 INFO     THURSDAY Apr 16
2020-04-11 22:16:11 INFO     		3 pm - 6 pm SOLD OUT	
2020-04-11 22:16:11 INFO     		6 pm - 9 pm SOLD OUT	
2020-04-11 22:16:11 INFO     		9 pm - 11 pm SOLD OUT	
2020-04-11 22:16:11 INFO     FRIDAY Apr 17
2020-04-11 22:16:11 INFO     		6 am - 9 am SOLD OUT	
2020-04-11 22:16:11 INFO     		9 am - 12 pm SOLD OUT	
2020-04-11 22:16:11 INFO     		3 pm - 6 pm SOLD OUT	
2020-04-11 22:16:11 INFO     		6 pm - 9 pm SOLD OUT	
2020-04-11 22:16:11 INFO     SATURDAY Apr 18
2020-04-11 22:16:11 INFO     		9 am - 12 pm SOLD OUT	
2020-04-11 22:16:12 INFO     		3 pm - 6 pm SOLD OUT	
2020-04-11 22:16:12 INFO     		6 pm - 9 pm SOLD OUT	
2020-04-11 22:16:27 INFO     200
2020-04-11 22:16:27 INFO     SUNDAY Apr 12
2020-04-11 22:16:28 INFO     MONDAY Apr 13
2020-04-11 22:16:28 INFO     		6 am - 9 am SOLD OUT	
2020-04-11 22:16:28 INFO     		9 am - 12 pm SOLD OUT	
2020-04-11 22:16:28 INFO     		3 pm - 6 pm SOLD OUT	
20

2020-04-11 22:17:39 INFO     		6 pm - 9 pm SOLD OUT	
2020-04-11 22:17:55 INFO     200
2020-04-11 22:17:55 INFO     SUNDAY Apr 12
2020-04-11 22:17:55 INFO     MONDAY Apr 13
2020-04-11 22:17:55 INFO     		6 am - 9 am SOLD OUT	
2020-04-11 22:17:55 INFO     		9 am - 12 pm SOLD OUT	
2020-04-11 22:17:55 INFO     		3 pm - 6 pm SOLD OUT	
2020-04-11 22:17:55 INFO     		6 pm - 9 pm SOLD OUT	
2020-04-11 22:17:55 INFO     TUESDAY Apr 14
2020-04-11 22:17:55 INFO     		6 am - 9 am SOLD OUT	
2020-04-11 22:17:55 INFO     		9 am - 12 pm SOLD OUT	
2020-04-11 22:17:55 INFO     		3 pm - 6 pm SOLD OUT	
2020-04-11 22:17:55 INFO     		6 pm - 9 pm SOLD OUT	
2020-04-11 22:17:55 INFO     WEDNESDAY Apr 15
2020-04-11 22:17:55 INFO     		6 am - 9 am SOLD OUT	
2020-04-11 22:17:55 INFO     		9 am - 12 pm SOLD OUT	
2020-04-11 22:17:55 INFO     		3 pm - 6 pm SOLD OUT	
2020-04-11 22:17:55 INFO     		6 pm - 9 pm SOLD OUT	
2020-04-11 22:17:55 INFO     THURSDAY Apr 16
2020-04-11 22:17:55 INFO     		3 pm - 6 pm SOLD OUT	
2